In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [10]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Add a channel dimension to the images (required for the CNN)
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)


In [11]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)


Epoch 1/5
1875/1875 [==============================] - 24s 12ms/step - loss: 0.1428 - accuracy: 0.9554
Epoch 2/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0464 - accuracy: 0.9854
Epoch 3/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0329 - accuracy: 0.9896
Epoch 4/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0243 - accuracy: 0.9922
Epoch 5/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0178 - accuracy: 0.9941


In [12]:
def generate_grad_cam(model, img, layer_name):
    # Get the model's output tensor
    output = model.output

    # Get the convolutional layer you want to visualize
    layer = model.get_layer(layer_name)

    # Calculate the gradient of the target class with regard to the output feature map of the selected layer
    grads = tf.gradients(output, layer.output)[0]

    # Average the gradients spatially
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Create a function to access the values of the intermediate layer and gradients
    iterate = keras.backend.function([model.input], [pooled_grads, layer.output[0]])

    # Get the pooled gradients and the output feature map of the selected layer
    pooled_grads_value, layer_output_value = iterate([img])

    # Multiply each channel in the feature map by "how important this channel is" with regard to the target class
    for i in range(layer_output_value.shape[-1]):
        layer_output_value[:, :, i] *= pooled_grads_value[i]

    # Average the weighted feature map
    heatmap = np.mean(layer_output_value, axis=-1)

    return heatmap


In [17]:
# Select an image from the test set
image_index = 0
test_image = test_images[image_index]

# Predict the class probabilities for the image
class_probabilities = model.predict(np.expand_dims(test_image, axis=0))

# Get the class with the highest probability
predicted_class = np.argmax(class_probabilities)

# Generate Grad-CAM heatmap for the predicted class
heatmap = generate_grad_cam(model, np.expand_dims(test_image, axis=0), 'conv2d')


1/1 [==============================] - 0s 21ms/step


RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [14]:
# Select an image from the test set
image_index = 0
test_image = test_images[image_index]

# Predict the class probabilities for the image
class_probabilities = model.predict(np.expand_dims(test_image, axis=0)

# Get the class with the highest probability
predicted_class = np.argmax(class_probabilities)

# Generate Grad-CAM heatmap for the predicted class
heatmap = generate_grad_cam(model, np.expand_dims(test_image, axis=0), 'conv2d_2')

SyntaxError: '(' was never closed (399497210.py, line 6)

In [ ]:
# Rescale the heatmap to be between 0 and 1
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

# Resize the heatmap to match the original image size
heatmap = cv2.resize(heatmap, (test_image.shape[1], test_image.shape[0]))
heatmap = np.uint8(255 * heatmap)

# Apply a heatmap color map
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Overlay the heatmap on the original image
superimposed_img = cv2.addWeighted(test_image.squeeze(), 0.6, heatmap, 0.4, 0)

# Display the original image, Grad-CAM heatmap, and the superimposed image
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(test_image.squeeze(), cmap='gray')
plt.title('Original Image')

plt.subplot(1, 3, 2)
plt.imshow(heatmap)
plt.title('Grad-CAM Heatmap')

plt.subplot(1, 3, 3)
plt.imshow(superimposed_img, cmap='jet')
plt.title('Superimposed Image')

plt.show()


In [19]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Add a channel dimension to the images (required for the CNN)
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

def generate_grad_cam(model, img, layer_name):
    with tf.GradientTape() as tape:
        # Get the model's output tensor
        output = model(img)
        class_idx = tf.argmax(output[0])
        # Get the convolutional layer you want to visualize
        layer = model.get_layer(layer_name)
        # Calculate the feature map
        target_layer_output = layer.output
    grads = tape.gradient(output, target_layer_output)[0]

    # Calculate the weights
    weights = tf.reduce_mean(grads, axis=(0, 1))

    # Get the feature map from the selected layer
    feature_map = model.get_layer(layer_name).output[0]

    # Generate the heatmap
    heatmap = tf.reduce_sum(tf.multiply(weights, feature_map), axis=-1)

    return heatmap

# Select an image from the test set
image_index = 0
test_image = test_images[image_index]

# Expand dimensions for batch size
test_image = np.expand_dims(test_image, axis=0)

# Generate Grad-CAM heatmap for the predicted class
heatmap = generate_grad_cam(model, test_image, 'conv2d_4')

# Rescale the heatmap to be between 0 and 1
heatmap = np.maximum(heatmap, 0)
heatmap /= tf.reduce_max(heatmap)

# Resize the heatmap to match the original image size
heatmap = tf.image.resize(heatmap, (28, 28))

# Convert to numpy array
heatmap = heatmap.numpy()

# Apply a heatmap color map
heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

# Overlay the heatmap on the original image
superimposed_img = cv2.addWeighted(test_image[0], 0.6, heatmap, 0.4, 0)

# Display the original image, Grad-CAM heatmap, and the superimposed image
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(test_image[0].squeeze(), cmap='gray')
plt.title('Original Image')

plt.subplot(1, 3, 2)
plt.imshow(heatmap.squeeze(), cmap='jet')
plt.title('Grad-CAM Heatmap')

plt.subplot(1, 3, 3)
plt.imshow(superimposed_img)
plt.title('Superimposed Image')

plt.show()


Epoch 1/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1424 - accuracy: 0.9561
Epoch 2/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0466 - accuracy: 0.9855
Epoch 3/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0320 - accuracy: 0.9902
Epoch 4/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0243 - accuracy: 0.9926
Epoch 5/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0182 - accuracy: 0.9942


ValueError: No such layer: conv2d_2. Existing layers are: ['conv2d_4', 'max_pooling2d_4', 'conv2d_5', 'max_pooling2d_5', 'flatten_2', 'dense_4', 'dense_5'].

In [21]:
# Generate Grad-CAM heatmap for the predicted class
heatmap = generate_grad_cam(model, test_image, 'conv2d_4')

# Rescale the heatmap to be between 0 and 1
heatmap = np.maximum(heatmap, 0)
heatmap /= tf.reduce_max(heatmap)

# Resize the heatmap to match the original image size
heatmap = tf.image.resize(heatmap, (28, 28))

# Convert to numpy array
heatmap = heatmap.numpy()

# Apply a heatmap color map
heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

# Overlay the heatmap on the original image
superimposed_img = cv2.addWeighted(test_image[0], 0.6, heatmap, 0.4, 0)

# Display the original image, Grad-CAM heatmap, and the superimposed image
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(test_image[0].squeeze(), cmap='gray')
plt.title('Original Image')

plt.subplot(1, 3, 2)
plt.imshow(heatmap.squeeze(), cmap='jet')
plt.title('Grad-CAM Heatmap')

plt.subplot(1, 3, 3)
plt.imshow(superimposed_img)
plt.title('Superimposed Image')

plt.show()


AttributeError: 'KerasTensor' object has no attribute '_id'